In [1]:
from cltk.corpus.readers import get_corpus_reader
latin_corpus = get_corpus_reader(corpus_name = 'latin_text_latin_library', language = 'latin')
from cltk.stem.lemma import LemmaReplacer
lemmatizer = LemmaReplacer('latin')

In [2]:
len(list(latin_corpus.docs()))

2141

In [3]:
files = latin_corpus.fileids()

In [4]:
livy_path = [file for file in files if 'livy' in file and 'per' not in file]

In [5]:
print(livy_path)

['livy/liv.1.txt', 'livy/liv.10.txt', 'livy/liv.2.txt', 'livy/liv.21.txt', 'livy/liv.22.txt', 'livy/liv.23.txt', 'livy/liv.24.txt', 'livy/liv.25.txt', 'livy/liv.26.txt', 'livy/liv.27.txt', 'livy/liv.28.txt', 'livy/liv.29.txt', 'livy/liv.3.txt', 'livy/liv.30.txt', 'livy/liv.31.txt', 'livy/liv.32.txt', 'livy/liv.33.txt', 'livy/liv.34.txt', 'livy/liv.35.txt', 'livy/liv.36.txt', 'livy/liv.37.txt', 'livy/liv.38.txt', 'livy/liv.39.txt', 'livy/liv.4.txt', 'livy/liv.40.txt', 'livy/liv.41.txt', 'livy/liv.42.txt', 'livy/liv.43.txt', 'livy/liv.44.txt', 'livy/liv.45.txt', 'livy/liv.5.txt', 'livy/liv.6.txt', 'livy/liv.7.txt', 'livy/liv.8.txt', 'livy/liv.9.txt', 'livy/liv.pr.txt']


In [6]:
livy_paras = latin_corpus.paras(livy_path)

In [7]:
livy_paras_list = list(livy_paras)

In [155]:
livy_words = latin_corpus.words(livy_path)

In [156]:
livy_words_list = list(livy_words)

In [157]:
print(livy_words_list[:10])

['Livy', ':', 'Book', 'I', 'TITI', 'LIVI', 'AB', 'VRBE', 'CONDITA', 'LIBER']


In [183]:
#this is for the tf-idf; I thought maybe I'd need to have the whole document?
from cltk.stop.latin import STOPS_LIST
from cltk.stem.lemma import LemmaReplacer

low_livy_list = [word.lower() for word in livy_words_list]
lemmatizer = LemmaReplacer('latin')
lematize_livy = [lemmatizer.lemmatize(word) for word in low_livy_list]
S = STOPS_LIST
flat_list = [item for sublist in lematize_livy for item in sublist]
livy_stops_removed = [w for w in flat_list if w not in STOPS_LIST]
junk = ['cn.', 't.', 'q.', "'", 'm.', 'p.', '[', ']', '.', ',', ' ', ':', ';', 'qui1', '-', 'que', '$', '%', '&','*','+', '-', '/', '<', '=', '>', '@', '^', '_',  '`', '{', '|', '}', '~', '?', '!', '«', '»']
livy_junk_removed = [w for w in livy_stops_removed if w not in junk]
clean_livy_words = livy_junk_removed

In [184]:
print(clean_livy_words[:10])

['livy', 'book', 'eo1', 'titi', 'livi', 'vrbe', 'condio', 'libo1', 'eo1', '1']


In [185]:
#print(livy_paras_list[:10])

In [186]:
len(livy_paras_list)

2209

In [187]:
#paras were wrapped in a list so need to go down one
livy_paras_flat = [item[0] for item in livy_paras_list]

In [188]:
len(livy_paras_flat)

2209

In [189]:
#print(livy_paras_flat[12])

In [190]:
len(livy_paras_flat)

2209

In [191]:
#now lower case everything
lower_livy_list = [[w.lower() for w in paras] for paras in livy_paras_flat]

In [192]:
from cltk.stem.lemma import LemmaReplacer

In [193]:
lemmatizer = LemmaReplacer('latin')

In [194]:
#lematize paragraphs
lematize_livy = [[lemmatizer.lemmatize(word, return_string = True) for word in paras] for paras in lower_livy_list]

In [195]:
#remove junk
junk = ['.', ',', ', ', ' ', ':', ';', 'qui1', '-', 'que', '$', '%', '&','*','+', '-', '/', '<', '=', '>', '@', '^', '_',  '`', '{', '|', '}', '~', '?', '!', '«', '»']
junk_removed = [[w for w in paras if w not in junk] for paras in lematize_livy]



In [196]:
#remove paragraphs that are only digits; rather return paragraphs that aren't only digits.
no_numbers = [[w for w in paras if w.isdigit() == False] for paras in junk_removed]

In [197]:
#print(no_numbers[:5])

In [198]:
#remove paragraphs that are shorter than 5 words
no_shorties = [para for para in no_numbers if len(para) >= 5]

In [199]:
len(no_shorties)

2120

In [200]:
#print(no_shorties[:5])

In [201]:
"""check how many words in the document - this doesn't seem right? But could have something to do with processing
paragraphs and lists?
"""

length_para = [len(para) for para in no_shorties]
#print(length_para)
sum(length_para)

54574

In [202]:
# give the finished product a better name
clean_livy = no_shorties

In [203]:
#print(clean_livy[:5])

In [204]:
count = 0 
for para in clean_livy:
    for word in para:
        if word == "praeda":
            count += 1
print(count)

30


In [210]:
#this is the function which may or may not be working. I doubt that there are only 30 paras with praeda for example ^^^
def includes(*keyword):
    count = 0
    for para in clean_livy:
        #for para in item:
            #if keyword in para:
            #if any([keyword in para]) == True:
        if any(word in para for word in keyword):
            count += 1
            
    print(str(count) + " out of " + str(len(no_shorties)) + " paragraphs")

In [211]:
includes("praeda","consul","sum1")

902 out of 2120 paragraphs


Checkpoint! Passed here is just tf-idf stuff which doesn't work.

In [139]:
import nltk
import numpy
import matplotlib
from nltk.tokenize import word_tokenize

In [144]:
mytext = nltk.Text(clean_livy)

In [146]:
from pprint import pprint
from gensim import corpora
from collections import defaultdict
from gensim.corpora import Dictionary

In [148]:
dictionary = corpora.Dictionary(mytext)

In [164]:
#print(dictionary.token2id)

In [153]:
doc = [[word for word in para] for para in mytext]

In [212]:
mydict = corpora.Dictionary([para for para in mytext])
corpus = [mydict.doc2bow(para for para in mytext)]